In [1]:
import pandas as pd

In [2]:
import requests as r
import json

In [5]:
with open("env.json", "r", encoding="utf-8") as file:
    config = json.load(file) 


In [6]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [7]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId='106'):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [8]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId='162',domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

In [9]:
daftar_data=getVarByDomain(domain='5100')
data=getDataByVarId(domain='5100',varId='162')
data

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",162,0,Tidak ada,125,2025,31,Triwulan I,1162012531,1162012531,1.87
1,1,"Pertanian, Kehutanan, dan Perikanan",162,0,Tidak ada,125,2025,32,Triwulan II,1162012532,1162012532,1.02
5,2,Pertambangan dan Penggalian,162,0,Tidak ada,125,2025,31,Triwulan I,2162012531,2162012531,0.75
6,2,Pertambangan dan Penggalian,162,0,Tidak ada,125,2025,32,Triwulan II,2162012532,2162012532,0.57
10,3,Industri Pengolahan,162,0,Tidak ada,125,2025,31,Triwulan I,3162012531,3162012531,1.28
...,...,...,...,...,...,...,...,...,...,...,...,...
85,18,Jumlah,162,0,Tidak ada,116,2016,31,Triwulan I,18162011631,18162011631,1.23
86,18,Jumlah,162,0,Tidak ada,116,2016,32,Triwulan II,18162011632,18162011632,0.51
87,18,Jumlah,162,0,Tidak ada,116,2016,33,Triwulan III,18162011633,18162011633,1.21
88,18,Jumlah,162,0,Tidak ada,116,2016,34,Triwulan IV,18162011634,18162011634,0.10


In [12]:
# proses cleaning
data = data[['verval_val','vervar_label','tahun_label','turtahun_label','content_label']]
data["vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# proses rename kolom
data_tb = data.rename(columns={
    'vervar_label'   : 'Lapangan_Usaha',
    'tahun_label'    : 'Tahun',
    'turtahun_label' : 'Triwulan',
    'content_label'  : 'Nilai'
})

# pivoting tabel (tambahkan Triwulan di columns)
data_tb = data_tb.pivot_table(
    index=['verval_val','Lapangan_Usaha'],
    columns=['Tahun','Triwulan'],
    values='Nilai'
)

# multi index jadikan 1 kolom
data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

# reset index
data_tb = data_tb.reset_index()


In [13]:
data_tb

,verval_val,Lapangan_Usaha,2016_Tahunan,2016_Triwulan I,2016_Triwulan II,2016_Triwulan III,2016_Triwulan IV,2017_Tahunan,2017_Triwulan I,2017_Triwulan II,...,2023_Triwulan II,2023_Triwulan III,2023_Triwulan IV,2024_Tahunan,2024_Triwulan I,2024_Triwulan II,2024_Triwulan III,2024_Triwulan IV,2025_Triwulan I,2025_Triwulan II
0,1,"Pertanian, Kehutanan, dan Perikanan",5.18,1.74,0.46,1.47,0.22,3.01,0.81,0.60,...,1.71,1.32,0.19,3.57,0.92,1.00,1.19,0.21,1.87,1.02
1,2,Pertambangan dan Penggalian,2.96,0.98,0.86,0.86,-0.10,2.09,-0.09,-0.47,...,0.99,0.09,1.58,3.24,1.05,0.18,0.16,1.91,0.75,0.57
2,3,Industri Pengolahan,4.20,1.19,0.31,0.51,1.02,3.13,1.00,0.63,...,0.53,1.77,0.23,3.68,1.31,0.50,1.00,0.84,1.28,0.91
3,4,Pengadaan Listrik dan Gas,18.49,1.50,3.81,7.16,0.67,17.26,7.33,4.85,...,0.15,-0.24,0.06,0.17,0.38,-0.01,-0.37,0.11,-0.03,1.03
4,5,"Pengadaan Air, Pengelolaan Sampah, Limbah dan ...",3.28,-0.02,0.38,3.23,2.79,5.08,0.50,0.44,...,0.15,-0.02,0.46,1.38,0.04,0.76,0.57,0.35,1.35,0.44
5,6,Konstruksi,2.72,0.44,0.13,0.33,0.24,2.55,0.76,0.58,...,-0.65,0.10,0.78,0.99,-0.03,0.28,0.17,0.88,0.63,0.17
6,7,Perdagangan Besar dan Eceran; Reparasi Mobil d...,2.58,0.69,0.56,0.86,0.46,3.63,1.33,1.03,...,0.32,0.06,0.29,1.42,0.20,0.83,0.04,0.92,0.75,1.39
7,8,Transportasi dan Pergudangan,4.90,0.02,0.74,4.96,-1.89,5.40,0.44,3.32,...,2.50,3.42,0.04,4.47,1.20,0.59,1.39,-0.76,-1.36,1.68
8,9,Penyediaan Akomodasi dan Makan Minum,2.56,1.47,0.54,0.25,0.28,3.41,0.98,1.69,...,0.47,0.66,1.14,4.41,1.27,1.37,0.92,1.05,0.44,0.39
9,10,Informasi dan Komunikasi,1.65,1.17,0.23,0.12,0.33,2.91,1.76,0.59,...,-0.05,0.04,1.30,1.82,0.19,0.36,0.86,-0.25,-0.03,0.02


In [14]:
data_tb.columns

Index(['verval_val', 'Lapangan_Usaha', '2016_Tahunan', '2016_Triwulan I',
       '2016_Triwulan II', '2016_Triwulan III', '2016_Triwulan IV',
       '2017_Tahunan', '2017_Triwulan I', '2017_Triwulan II',
       '2017_Triwulan III', '2017_Triwulan IV', '2018_Tahunan',
       '2018_Triwulan I', '2018_Triwulan II', '2018_Triwulan III',
       '2018_Triwulan IV', '2019_Tahunan', '2019_Triwulan I',
       '2019_Triwulan II', '2019_Triwulan III', '2019_Triwulan IV',
       '2020_Tahunan', '2020_Triwulan I', '2020_Triwulan II',
       '2020_Triwulan III', '2020_Triwulan IV', '2021_Tahunan',
       '2021_Triwulan I', '2021_Triwulan II', '2021_Triwulan III',
       '2021_Triwulan IV', '2022_Tahunan', '2022_Triwulan I',
       '2022_Triwulan II', '2022_Triwulan III', '2022_Triwulan IV',
       '2023_Tahunan', '2023_Triwulan I', '2023_Triwulan II',
       '2023_Triwulan III', '2023_Triwulan IV', '2024_Tahunan',
       '2024_Triwulan I', '2024_Triwulan II', '2024_Triwulan III',
       '2024_Triwul

In [15]:
data_tb.to_excel("data_162.xlsx", sheet_name="PivotTriwulan", index=False)
